In [2]:
import dotenv

dotenv.load_dotenv()

True

In [3]:
from langchain_openai import ChatOpenAI, AzureChatOpenAI

#chat = ChatOpenAI(model="gpt-3.5-turbo-1106")
chat = AzureChatOpenAI(
    openai_api_version="2024-02-15-preview",
    azure_deployment="gpt-35-turbo",
    temperature=0.2)

In [4]:
from langchain_community.document_loaders import DirectoryLoader
loader = DirectoryLoader('./docs', glob="**/*.md")
data = loader.load()
len(data)

2

In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20)
all_splits = text_splitter.split_documents(data)

print(len(all_splits))

39


In [6]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)



modell = "paraphrase-multilingual-MiniLM-L12-v2"

embedding_function = SentenceTransformerEmbeddings(model_name=modell)
vectorstore = Chroma.from_documents(documents=all_splits, embedding=embedding_function)

retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

In [8]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

SYSTEM_TEMPLATE = """
Answer the user's questions based on the below context.
If the context doesn't contain any relevant information to the question, don't make something up and just say "I don't know":

<context>
{context}
</context>
"""

question_answering_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            SYSTEM_TEMPLATE,
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

document_chain = create_stuff_documents_chain(chat, question_answering_prompt)

In [9]:
from typing import Dict

from langchain_core.runnables import RunnablePassthrough

def parse_retriever_input(params: Dict):
    return params["messages"][-1].content


retrieval_chain = RunnablePassthrough.assign(
    context=parse_retriever_input | retriever,
).assign(
    answer=document_chain,
)

In [13]:
from langchain_core.messages import HumanMessage, AIMessage

retrieval_chain.invoke(
    {
        "messages": [
            HumanMessage(content="Wer ist die Gechäftsführerin der MMS?")
        ],
    }
)

{'messages': [HumanMessage(content='Wer ist die Gechäftsführerin der MMS?')],
 'context': [Document(page_content='des Unternehmens. Ehemalige Geschäftsführer sind:', metadata={'source': 'docs/mms.md'}),
  Document(page_content='Personen\n\nOliver Guhr, Schlungsleiter', metadata={'source': 'docs/index.md'}),
  Document(page_content='und IT-Sicherheit.', metadata={'source': 'docs/mms.md'}),
  Document(page_content='Indem T-Systems MMS eine künstliche Intelligenz mit einer weiblichen Gesangsstimme trainierte, war das Unternehmen 2022 maßgeblich an der weltweit ersten KI-Oper „chasing waterfalls“ beteiligt, die', metadata={'source': 'docs/mms.md'}),
  Document(page_content='2018 schloss T-Systems MMS einen Kooperationsvertrag mit dem Smart Systems Hub und ist seitdem einer der strategischen Partner. Mit dem Ziel, die reale und die digitale Welt zu vernetzen, ermöglicht', metadata={'source': 'docs/mms.md'})],
 'answer': 'Die Information über die Geschäftsführerin der MMS ist nicht im gegebe

In [14]:
retrieval_chain.invoke(
    {
        "messages": [
            HumanMessage(content="Was ist die Telekom MMS?"),
            AIMessage(
                content="Die Telekom MMS GmbH (Telekom MMS) ist ein Digital-Dienstleister, der sich als Begleiter von Großkonzernen und mittelständischen Unternehmen bei der digitalen Transformation versteht. Sie entwickelt zukunftsfähige Geschäftsmodelle für digitale Erlebnisse und bietet dynamisches Web- und Application-Management."
            ),
            HumanMessage(content="Erzähl mir mehr!"),
        ],
    }
)

{'messages': [HumanMessage(content='Was ist die Telekom MMS?'),
  AIMessage(content='Die Telekom MMS GmbH (Telekom MMS) ist ein Digital-Dienstleister, der sich als Begleiter von Großkonzernen und mittelständischen Unternehmen bei der digitalen Transformation versteht. Sie entwickelt zukunftsfähige Geschäftsmodelle für digitale Erlebnisse und bietet dynamisches Web- und Application-Management.'),
  HumanMessage(content='Erzähl mir mehr!')],
 'context': [Document(page_content='des Unternehmens. Ehemalige Geschäftsführer sind:', metadata={'source': 'docs/mms.md'}),
  Document(page_content='Personen\n\nOliver Guhr, Schlungsleiter', metadata={'source': 'docs/index.md'}),
  Document(page_content='Geschichte', metadata={'source': 'docs/mms.md'}),
  Document(page_content='und IT-Sicherheit.', metadata={'source': 'docs/mms.md'}),
  Document(page_content='über Content-, Test- und Support-Services bis zur Entwicklung von Software. Dazu zählen Web- und Applikations-Management und das Test and Inte

## Wie lösen wir das Problem?

In [29]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableBranch

#query_transforming_retriever_chain = RunnableBranch( 
#    # If messages, then we pass inputs to LLM chain to transform the query, then pass to retriever
#    query_transform_prompt | chat | StrOutputParser() | retriever,
#).with_config(run_name="chat_retriever_chain")
query_transform_prompt = ChatPromptTemplate.from_messages(
    [
        MessagesPlaceholder(variable_name="messages"),
        (
            "user",
            "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation. Only respond with the query, nothing else.",
        ),
    ]
)

query_transforming_retriever_chain = RunnableBranch(
    (
        lambda x: len(x.get("messages", [])) == 0,
        # If only one message, then we just pass that message's content to retriever
        (lambda x: x["messages"][-1].content) | retriever,
    ),
    # If messages, then we pass inputs to LLM chain to transform the query, then pass to retriever
    query_transform_prompt | chat | StrOutputParser() | retriever,
).with_config(run_name="chat_retriever_chain")

conversational_retrieval_chain = RunnablePassthrough.assign(
    context=query_transforming_retriever_chain,
).assign(
    answer=document_chain,
)

In [32]:
conversational_retrieval_chain.invoke(
    {
        "messages": [
            HumanMessage(content="Wer ist die Gechäftsführerin der MMS?"),
        ]
    }
)

{'messages': [HumanMessage(content='Wer ist die Gechäftsführerin der MMS?')],
 'context': [Document(page_content='des Unternehmens. Ehemalige Geschäftsführer sind:', metadata={'source': 'docs/mms.md'}),
  Document(page_content='Die Geschäftsführung besteht aus einer Doppelspitze: Ralf Pechmann, mit Hauptverantwortung im Bereich Vertrieb, Marketing, Kunden und Marcus Gaube als kaufmännischer Leiter des Unternehmens.', metadata={'source': 'docs/mms.md'}),
  Document(page_content='Die Deutsche Telekom MMS GmbH (Telekom MMS) ist ein Digital-Dienstleister, der sich als Begleiter von Großkonzernen und mittelständischen Unternehmen bei der digitalen Transformation versteht.', metadata={'source': 'docs/mms.md'}),
  Document(page_content='Personen\n\nOliver Guhr, Schlungsleiter', metadata={'source': 'docs/index.md'}),
  Document(page_content='2022 wurde T-Systems MMS erstmalig in die Lünendonk-Liste der führenden Anbieter im Segment Digital Experience Services (DXS) sowie im Sub-Ranking der Unt

In [39]:
#import langchain
#langchain.verbose = False
#langchain.debug = False

conversational_retrieval_chain.invoke(
    {
        "messages": [
            HumanMessage(content="Was ist die Telekom MMS?"),
            AIMessage(
                content="Die Telekom MMS GmbH (Telekom MMS) ist ein Digital-Dienstleister, der sich als Begleiter von Großkonzernen und mittelständischen Unternehmen bei der digitalen Transformation versteht. Sie entwickelt zukunftsfähige Geschäftsmodelle für digitale Erlebnisse und bietet dynamisches Web- und Application-Management."
            ),
            HumanMessage(content="Erzähl mir mehr!"),
        ],
    }
)

{'messages': [HumanMessage(content='Was ist die Telekom MMS?'),
  AIMessage(content='Die Telekom MMS GmbH (Telekom MMS) ist ein Digital-Dienstleister, der sich als Begleiter von Großkonzernen und mittelständischen Unternehmen bei der digitalen Transformation versteht. Sie entwickelt zukunftsfähige Geschäftsmodelle für digitale Erlebnisse und bietet dynamisches Web- und Application-Management.'),
  HumanMessage(content='Erzähl mir mehr!')],
 'context': [Document(page_content='Die Deutsche Telekom MMS GmbH (Telekom MMS) ist ein Digital-Dienstleister, der sich als Begleiter von Großkonzernen und mittelständischen Unternehmen bei der digitalen Transformation versteht.', metadata={'source': 'docs/mms.md'}),
  Document(page_content='Telekom MMS', metadata={'source': 'docs/mms.md'}),
  Document(page_content='Geschäftskunden der Telekom Deutschland heran und kann den Konzern auf dem Weg zur „Leading Digital Telko“ unterstützen.', metadata={'source': 'docs/mms.md'}),
  Document(page_content='Üb

In [46]:
from langchain.callbacks import get_openai_callback
with get_openai_callback() as cb:
    data = conversational_retrieval_chain.invoke(
        {
            "messages": [
                HumanMessage(content="Was ist die Telekom MMS?"),
                AIMessage(
                    content="Die Telekom MMS GmbH (Telekom MMS) ist ein Digital-Dienstleister, der sich als Begleiter von Großkonzernen und mittelständischen Unternehmen bei der digitalen Transformation versteht. Sie entwickelt zukunftsfähige Geschäftsmodelle für digitale Erlebnisse und bietet dynamisches Web- und Application-Management."
                ),
                HumanMessage(content="Erzähl mir mehr!"),
            ],
        }
    )
    print(data["answer"])
    print(cb)
    #print(f"Total Cost (USD): ${format(cb.total_cost, '.6f')}")

Die Telekom MMS GmbH ist ein Unternehmen, das Geschäftskunden der Telekom Deutschland bei der digitalen Transformation unterstützt. Mit über 2.200 Mitarbeitenden an neun Standorten in Deutschland entwickelt sie zukunftsfähige Geschäftsmodelle für digitale Erlebnisse und bietet Web- und Application-Management. Im Jahr 2018 schloss T-Systems MMS einen Kooperationsvertrag mit dem Smart Systems Hub und ist seitdem einer der strategischen Partner. Ihr Ziel ist es, die reale und die digitale Welt zu vernetzen.
Tokens Used: 621
	Prompt Tokens: 488
	Completion Tokens: 133
Successful Requests: 2
Total Cost (USD): $0.000998


<function dict.values>